# RAG简单工作流

## 环境

python 3.12  (>3.10就好)
torch == 2.6.0 (建议)
transformers >= 4.52.0 (建议)
hnswlib 

## 文本数据格式

json列表格式, 如下：

[
    str1,
    str2,
    str3,
    ...
]

## 问答输入

格式也是可以是`list[str]`，也可以是`str`，当是`list[str]`时，默认推理模型的`batch_size`为4

## 推理输出

`list[dict[str,Any]]`

[
    {
        'query': <输入进来的问题>,
        'response': <最终模型输出的答案>,
        'search_result':<模型检索向量库时产生的结果>
    },
    ...
]


## 指令模板样式

"请依据所给的问题以及文本材料，生成最合适的答案。\n\n问题:\n{query}\n\n文本材料:\n{evidences}\n\n"

In [1]:
import os, re, argparse
# os.environ["CUDA_VISIBLE_DEVICES"] = "1" # 这里替换成你想要的GPU序号

from Components import *

# 配置模型参数
config=dict(
    embedding_model_or_path="J:\\我的博士论文\\LLMs\\BAAI\\bge-m3",  # 你自己的文本嵌入模型地址
    embedding_text_source="data\\Text.json",       # 用于检索的文本数据源
    embedding_gpu="cuda:0",   # 其实如果初次嵌入好了之后就没必要再用GPU了，相似度计算什么的都可以跑在CPU上
    embedding_batch_size=16,   # 模型进行嵌入操作的时候的批次大小
    inference_model_or_path="J:\\我的博士论文\\LLMs\\Qwen\\Qwen2.5-Coder-7B-Instruct",  # 你自己的推理模型地址
    inference_gpu="cuda:1",   # 如果使用巨TM大的LLM推理，这里直接填`cuda`就好 （如果是其他环境就按照其他环境的参数写，比如华为的填写`npu`）
    QInt4=False             # 是否使用QInt4量化（使用的话用于减少显存占用）
)
# 创建RAG工作流
rag_worlflow=SimpleRAG_Workflow(**config)


c:\ProgramData\anaconda3\envs\llmplus\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


hnsw init done, shape = (340273, 1024)


Loading checkpoint shards: 100%|██████████| 4/4 [00:11<00:00,  2.87s/it]


## 运行

In [2]:
gen_kwargs=dict(max_new_tokens=2048,batch_size=4)
prompt_prefix="请依据所给的问题以及文本材料，生成最合适的答案。\n\n问题:\n{query}\n\n文本材料:\n{evidences}\n\n"
qas=load_from_json("data\\qas.json")
results=rag_worlflow.query(prompt_prefix,[n['问题'] for n in qas[:10]],10,**gen_kwargs)
results

响应中...: 100%|██████████| 3/3 [01:06<00:00, 22.26s/it]


[{'query': '不需要投保“交强险”的车辆的车船税由谁负责征收？',
  'response': '不需要投保“交强险”的车辆的车船税由地方税务机关负责征收。',
  'search_result': {'query': '不需要投保“交强险”的车辆的车船税由谁负责征收？',
   'retrieve_results': ['（内容来自:  山东省青岛市地方税务局关于车船税征收管理的通告 ）  三、 从2007年8月1日起，青岛市对属于投保交强险车辆全部由各保险机构在销售交强险时代收代缴车船税。根据《机动车交通事故责任强制保险条例》规定，对不需要投保交强险，但属于车船税征税范围的车辆和船舶，并在车船管理部门登记的车船，由地方税务机关征收车船税。',
    '（内容来自:  国家税务总局关于车船税征管若干问题的公告 ）  五、 关于扣缴义务人代收代缴后车辆登记地主管税务机关不再征收车船税 纳税人在购买“交强险”时，由扣缴义务人代收代缴车船税的，凭注明已收税款信息的“交强险”保险单，车辆登记地的主管税务机关不再征收该纳税年度的车船税。再次征收的，车辆登记地主管税务机关应予退还。',
    '（内容来自:  国家税务总局关于发布《车船税管理规程（试行）》的公告 ）  第十一条 对已经向主管税务机关申报缴纳车船税的纳税人，保险机构在销售机动车第三者责任强制保险时，不再代收车船税，但应当根据纳税人的完税凭证原件，将车辆的完税凭证号和出具该凭证的税务机关名称录入交强险业务系统。对出具税务机关减免税证明的车辆，保险机构在销售机动车第三者责任强制保险时，不代收车船税，保险机构应当将减免税证明号和出具该证明的税务机关名称录入交强险业务系统。纳税人对保险机构代收代缴税款数额有异议的，可以直接向税务机关申报缴纳，也可以在保险机构代收代缴税款后向税务机关提出申诉，税务机关应在接到纳税人申诉后按照本地区代收代缴管理办法规定的受理程序和期限进行处理。',
    '（内容来自:  国家税务总局关于车船税征管若干问题的通知 ）  为方便车船税征缴，进一步提高征管质量和效率，切实做好车船税的征收工作，根据《中华人民共和国车船税暂行条例》（以下简称“条例”）及其实施细则的有关规定，现就车船税征管有关问题通知如下：\ue004 一、 关于不在车辆登记地购买保险代收代缴车船税问题 在一个纳税